# Day 7

## Day 7.1

## Approach 1: Create a single expression by recursive substitution, then evaluate!

In [20]:
binary_command = {'NOT': '~', 'AND': '&', 'OR': '|', 'LSHIFT': '<<', 'RSHIFT': '>>'}
operators = binary_command.values()

In [21]:
import csv

def translate(l):
    return [binary_command[a] if a in binary_command else a for a in l]

def display(input_file):

    """produce a dict mapping variables to expressions"""
    
    commands = []
    with open(input_file, 'rt') as f_input:
        csv_reader = csv.reader(f_input, delimiter=' ')
        for line in csv_reader:
            commands.append((line[-1], ' '.join(list(translate(line[:-2])))))
    return dict(commands)

In [32]:
import re

def extract_variables(expr):
    varbls = []
    regex_pattern = '\s|\\)|\\('
    l = re.split(regex_pattern, expr)
    for a in l:
        if (a not in operators) and (not a.isnumeric()) and (a != ''):
            varbls.append(a)
    return set(varbls)


def create_instance(wire):
    exec_python = commands[wire]
    pending = extract_variables(commands[wire])
    count = 0
    while pending and (count < 200):
        s = pending.pop()
        expr = commands[s]
        exec_python = re.sub('({0})'.format(s), '( {0} )'.format(expr), exec_python)
        pending = pending.union(extract_variables(exec_python))
        count += 1
    return wire + ' = ' + exec_python

def evaluate(var):
    instance = create_instance(var)
    exec(instance)
    return np.uint16(locals()[var])

### Test

In [40]:
commands = display('inputs/input7.test.txt')

In [41]:
def test():
    assert(evaluate('d') == 72)
    assert(evaluate('e') == 507)
    assert(evaluate('f') == 492)
    assert(evaluate('g') == 114)
    assert(evaluate('h') == 65412)
    assert(evaluate('i') == 65079)
    assert(evaluate('x') == 123)
    assert(evaluate('y') == 456)

In [42]:
test()

This approach seems correct, but it creates huge expressions along the way that become harder and harder to parse. Thus the time to a final expression that wraps up all the computations is very long. Two ideas to carry on: i) concurrent evaluation of expressions; ii) define lazy variables/functions that collect all the dependencies of the circuit and start firing upon request.

## Approach 2: Concurrent evaluation from known variables.

The solution provided hereto owes credit to this source: https://www.reddit.com/r/adventofcode/comments/5id6w0/2015_day_7_part_1_python_wrong_answer/

In [21]:
import numpy as np

def RSHIFT(a, b):
    result = np.uint16(a) >> int(b)
    return int(result)

def LSHIFT(a, b):
    result = np.uint16(a) << int(b)
    return int(result)

def OR(a, b):
    result = np.uint16(a) | np.uint16(b)
    return int(result)

def AND(a, b):
    result = np.uint16(a) & np.uint16(b)
    return int(result)

def NOT(a):
    result = ~ np.uint16(a)
    return int(result)

In [28]:
import csv
def display(input_file):

    """produce a dict mapping variables to expressions"""
    
    commands = []
    with open(input_file, 'rt') as f_input:
        csv_reader = csv.reader(f_input, delimiter=' ')
        for line in csv_reader:
            commands.append((line[-1], line[:-2]))
    return dict(commands)

In [105]:
def evaluate(wire):
    known = {}
    while wire not in known:
        if wire in known:
            break
        for k, v in commands.items():
            if (len(v) == 1) and (v[0].isnumeric()) and (k not in known):
                known[k] = int(v[0])
            elif (len(v) == 1) and (v[0] in known) and (k not in known):
                known[k] = known[v[0]]
            elif ('AND' in v) and (v[0] in known) and (v[2] in known):
                known[k] = AND(known[v[0]], known[v[2]])
            elif ('AND' in v) and (v[0].isnumeric()) and (v[2] in known):
                known[k] = AND(int(v[0]), known[v[2]])
            elif ('AND' in v) and (v[0] in known) and (v[2].isnumeric()):
                known[k] = AND(known[v[0]], int(v[2]))
            elif ('OR' in v) and (v[0] in known) and (v[2] in known):
                known[k] = OR(known[v[0]], known[v[2]])
            elif ('OR' in v) and (v[0].isnumeric()) and (v[2] in known):
                known[k] = OR(int(v[0]), known[v[2]])
            elif ('OR' in v) and (v[0] in known) and (v[2].isnumeric()):
                known[k] = OR(known[v[0]], int(v[2]))
            elif ('LSHIFT' in v) and (v[0] in known):
                known[k] = LSHIFT(known[v[0]], v[2])
            elif ('RSHIFT' in v) and (v[0] in known):
                known[k] = RSHIFT(known[v[0]], v[2])
            elif ('NOT' in v) and (v[1] in known):
                known[k] = NOT(known[v[1]])
    return known[wire]

### Test 0

In [107]:
commands = display('inputs/input7.test1.txt')
commands

{'a': ['b'], 'b': ['1', 'OR', 'x'], 'x': ['12']}

In [108]:
evaluate('a')

13

### Test 1

In [109]:
commands = display('inputs/input7.test2.txt')
commands

{'d': ['x', 'AND', 'y'],
 'e': ['x', 'OR', 'y'],
 'f': ['x', 'LSHIFT', '2'],
 'g': ['y', 'RSHIFT', '2'],
 'h': ['NOT', 'x'],
 'i': ['NOT', 'y'],
 'x': ['123'],
 'y': ['456']}

In [110]:
test()

### Solution

In [111]:
commands = display('inputs/input7.txt')
evaluate('a')

16076

## Approach 3: With Lazy Variable Wrapper (Python)

In [3]:
import csv
import numpy as np

def display(input_file):

    """produce a dict mapping variables to expressions"""
    
    commands = []
    with open(input_file, 'rt') as f_input:
        csv_reader = csv.reader(f_input, delimiter=' ')
        for line in csv_reader:
            commands.append((line[-1], line[:-2]))
    return dict(commands)

In [2]:
class LazyVar(object):
    def __init__(self, func):
        self.func = func
        self.value = None
    def __call__(self):
        if self.value is None:
            self.value = self.func()
        return self.value

In [3]:
binary_command = {'NOT': '~', 'AND': '&', 'OR': '|', 'LSHIFT': '<<', 'RSHIFT': '>>'}

def translate(l):
    translated = []
    for a in l:
        if a in binary_command:
            b = binary_command[a]
        elif a.isnumeric():
            b = 'np.uint16({})'.format(a)
        else:
            b = '{}.func()'.format('var_' + a)
        translated.append(b)
    return translated

### Test

In [4]:
commands = display('inputs/input7.test2.txt')

In [5]:
commands = display('inputs/input7.test2.txt')
for k, v in commands.items():
    command_str = '{0} = LazyVar(lambda: {1})'.format('var_' + k, ''.join(translate(v)))
    print(command_str)
    exec(command_str)

var_y = LazyVar(lambda: np.uint16(456))
var_d = LazyVar(lambda: var_x.func()&var_y.func())
var_i = LazyVar(lambda: ~var_y.func())
var_h = LazyVar(lambda: ~var_x.func())
var_e = LazyVar(lambda: var_x.func()|var_y.func())
var_f = LazyVar(lambda: var_x.func()<<np.uint16(2))
var_x = LazyVar(lambda: np.uint16(123))
var_g = LazyVar(lambda: var_y.func()>>np.uint16(2))


In [6]:
def test():
    assert(var_d.func() == 72)
    assert(var_e.func() == 507)
    assert(var_f.func() == 492)
    assert(var_g.func() == 114)
    assert(var_h.func() == 65412)
    assert(var_i.func() == 65079)
    assert(var_x.func() == 123)
    assert(var_y.func() == 456)

In [7]:
test()

Although the approach passes the test, it does not end in reasonable time for the full input.

## Approach 4: With Lazy Evaluation in R

The approach now is to exploit the lazy evaluation capabilities in R. So we leverage Python to create an R script that does the job.

In [35]:
def rscript_command(var, l):
    vocab =  {'AND'    : 'bitwAnd', 
              'OR'     : 'bitwOr',
              'LSHIFT' : 'bitwShiftL',
              'RSHIFT' : 'bitwShiftR'}
    if len(l) == 3:
        func = vocab[l[1]]
        arg1 = l[0] if l[0].isdigit() else 'var_' + l[0] + '()'
        arg2 = l[2] if l[2].isdigit() else 'var_' + l[2] + '()'
        return 'var_{0} <- function(a={1}, b={2})'.format(var, arg1, arg2) + ' {' + '{0}(a,b)'.format(func) + '}'
    elif len(l) == 2:
        func = 'bitwNot'
        arg1 = l[1] if l[1].isdigit() else 'var_' + l[1] + '()'
        return 'var_{0} <- function(a={1})'.format(var, arg1) + ' {' + '{0}(a)'.format(func) + '}'
    else:
        arg1 = l[0] if l[0].isdigit() else 'var_' + l[0] + '()'
        return 'var_{0} <- function(a={1})'.format(var, arg1) + ' {' + 'a' + '}'

def generate_rscript(commands, target):
    with open('day7_commands.R', 'wt') as f:
        for k, v in commands.items():
            f.write(rscript_command(k, v)+'\n')
        f.write('var_' + target + '()')

### Test

In [36]:
commands = display('inputs/input7.test2.txt')
generate_rscript(commands, 'd')

In [37]:
! cat day7_commands.R

var_g <- function(a=var_y(), b=2) {bitwShiftR(a,b)}
var_x <- function(a=123) {a}
var_e <- function(a=var_x(), b=var_y()) {bitwOr(a,b)}
var_f <- function(a=var_x(), b=2) {bitwShiftL(a,b)}
var_i <- function(a=var_y()) {bitwNot(a)}
var_d <- function(a=var_x(), b=var_y()) {bitwAnd(a,b)}
var_y <- function(a=456) {a}
var_h <- function(a=var_x()) {bitwNot(a)}
var_d()

In [38]:
!Rscript day7_commands.R

[1] 72


### Solution

In [39]:
commands = display('inputs/input7.txt')
generate_rscript(commands, 'a')

In [40]:
! cat day7_commands.R 

var_ak <- function(a=var_x(), b=var_ai()) {bitwAnd(a,b)}
var_ck <- function(a=var_ci(), b=3) {bitwShiftR(a,b)}
var_ed <- function(a=var_ea(), b=var_eb()) {bitwAnd(a,b)}
var_li <- function(a=var_lf(), b=5) {bitwShiftR(a,b)}
var_ce <- function(a=var_bk(), b=1) {bitwShiftL(a,b)}
var_co <- function(a=var_cn()) {bitwNot(a)}
var_fq <- function(a=var_fo(), b=3) {bitwShiftR(a,b)}
var_l <- function(a=var_d(), b=var_j()) {bitwAnd(a,b)}
var_et <- function(a=var_er(), b=var_es()) {bitwOr(a,b)}
var_ik <- function(a=var_ih(), b=var_ij()) {bitwAnd(a,b)}
var_bi <- function(a=1, b=var_bh()) {bitwAnd(a,b)}
var_bu <- function(a=var_br(), b=var_bt()) {bitwAnd(a,b)}
var_kg <- function(a=var_jm(), b=1) {bitwShiftL(a,b)}
var_ih <- function(a=var_ia(), b=var_ig()) {bitwOr(a,b)}
var_hg <- function(a=var_he(), b=3) {bitwShiftR(a,b)}
var_kx <- function(a=var_kk(), b=var_kv()) {bitwAnd(a,b)}
var_ae <- function(a=var_ab(), b=var_ad()) {bitwAnd(a,b)}
var_iy <- function(a=var_iw(), b=var_ix()) {bitwOr(a,b)}
var_dm <

In [ ]:
!Rscript day7_commands.R

Although this approach is more natural than defining a LazyWrapper in Python, it takes quite a lot of time to execute, so this is not a very cool solution after all.

## Day 7.2

In [114]:
commands = display('inputs/input7.txt')
commands['b'] = ['16076']
evaluate('a')

2797